# Data Cleaning & Manipulation

**\---**  

**title: Cyclistic Data Analysis Case Study**

**author: Sibeso Like**

**date: July 21, 2025**

**\---**

Before conducting meaningful analysis, the raw data underwent a comprehensive cleaning process to ensure accuracy, consistency, and reliability. This step was essential for handling missing values, correcting data types, standardizing station IDs, and generating new features such as ride duration, day of the week, and hour of ride start. All cleaning operations were executed using **SQL Server**, following best practices for data preprocessing to prepare the dataset for accurate analysis and visualization.

## Master Table Structure
The master table, `master_dataset`, was created to merge all 12 datasets. Below is its structure:

In [ ]:
-- Master table structure
COLUMN_NAME             DATA_TYPE       CHARACTER_MAXIMUM_LENGTH    IS_NULLABLE
ride_id                varchar         75                         NO
rideable_type          varchar         255                        YES
started_at             smalldatetime   NULL                       YES
ended_at               smalldatetime   NULL                       YES
start_station_name     varchar         255                        YES
start_station_id       varchar         40                         YES
end_station_name       varchar         255                        YES
end_station_id         varchar         100                        YES
start_lat              decimal         NULL                       YES
start_lng              decimal         NULL                       YES
end_lat                decimal         NULL                       YES
end_lng                decimal         NULL                       YES
member_casual          varchar         20                         NO

## Data Preview
To inspect the data, the first 1000 rows of the master table were selected.

In [ ]:
SELECT TOP (1000) [ride_id]
      ,[rideable_type]
      ,[started_at]
      ,[ended_at]
      ,[start_station_name]
      ,[start_station_id]
      ,[end_station_name]
      ,[end_station_id]
      ,[start_lat]
      ,[start_lng]
      ,[end_lat]
      ,[end_lng]
      ,[member_casual]
  FROM [Cyclistic_Case_Study].[dbo].[master_dataset];

## Data Import and Consolidation
Each dataset was inserted into the `master_dataset` table, ensuring no duplicate `ride_id` entries.

In [ ]:
INSERT INTO master_dataset
SELECT *
FROM dataset_1
WHERE ride_id NOT IN (
    SELECT ride_id FROM master_dataset
);

### Handling Import Issues for October 2024 Dataset
The `202410-divvy-tripdata` dataset had data entry errors, preventing standard import. A `BULK INSERT` was used, and affected columns were temporarily altered to `VARCHAR` to accommodate the data.

In [ ]:
BULK INSERT dataset_4
FROM 'C:\Users\mmnja\OneDrive\Documents\My Docs\DATA ANALYTICS\Google Sheet CSV Files\202410-divvy-tripdata.csv'
WITH (
    FIELDTERMINATOR = ',',
    ROWTERMINATOR = '\n',
    FIRSTROW = 2, 
    TABLOCK
);

ALTER TABLE dataset_4
ALTER COLUMN started_at VARCHAR(255) null;
ALTER TABLE dataset_4
ALTER COLUMN ended_at VARCHAR(255);
ALTER TABLE dataset_4
ALTER COLUMN end_lng VARCHAR(255) null;

### Cleaning `dataset_4`
Quotes were removed from specific columns, and the columns were converted back to their original data types before inserting into the master table.

In [ ]:
UPDATE dataset_4
SET ride_id = REPLACE(TRIM(ride_id), '"', '');
GO
UPDATE dataset_4
SET rideable_type = REPLACE(TRIM(rideable_type), '"', '');
GO
UPDATE dataset_4
SET started_at = REPLACE(TRIM(started_at), '"', '');
GO
UPDATE dataset_4
SET ended_at = REPLACE(TRIM(ended_at), '"', '');
GO
UPDATE dataset_4
SET member_casual = REPLACE(TRIM(member_casual), '"', '');

ALTER TABLE dataset_4
ALTER COLUMN started_at smalldatetime;
ALTER TABLE dataset_4
ALTER COLUMN ended_at smalldatetime;
ALTER TABLE dataset_4
ALTER COLUMN end_lng DECIMAL(16,13);

INSERT INTO master_dataset
SELECT *
FROM dataset_4
WHERE ride_id NOT IN (
    SELECT ride_id FROM master_dataset
);

## Data Cleaning in Master Dataset
With all datasets consolidated, cleaning was performed on the `master_dataset`.

### Removing Duplicates
Duplicate `ride_id` entries were removed using a CTE.

In [ ]:
WITH RankedDuplicates AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY ride_id ORDER BY (SELECT NULL)) AS rn
    FROM master_dataset
)
DELETE FROM RankedDuplicates
WHERE rn > 1;

### Setting Primary Key
The `ride_id` column was set as the primary key.

In [ ]:
ALTER TABLE master_dataset
ADD CONSTRAINT PK_master_dataset_ride_id PRIMARY KEY (ride_id);

### Adding Indexes
Indexes were created to optimize query performance for filtering and grouping.

In [ ]:
CREATE INDEX idx_started_at ON master_dataset(started_at);
CREATE INDEX idx_ended_at ON master_dataset(ended_at);
GO
CREATE INDEX idx_start_station_id ON master_dataset(start_station_id);
CREATE INDEX idx_end_station_id ON master_dataset(end_station_id);
GO
CREATE INDEX idx_member_casual ON master_dataset(member_casual);

### Removing NULL Values
Rows with NULL values in critical columns were deleted.

In [ ]:
DELETE FROM master_dataset
WHERE started_at IS NULL
   OR ended_at IS NULL
   OR start_station_name IS NULL
   OR start_station_id IS NULL
   OR end_station_name IS NULL
   OR end_station_id IS NULL;

### Validating Date Ranges
The earliest and latest dates in `started_at` and `ended_at` were checked for anomalies.

In [ ]:
SELECT MIN(started_at) AS EarliestDate, MAX(started_at) AS LatestDate FROM master_dataset;
SELECT MIN(ended_at) AS EarliestDate, MAX(ended_at) AS LatestDate FROM master_dataset;
SELECT * FROM master_dataset WHERE started_at='1900-01-01';
SELECT * FROM master_dataset WHERE ended_at ='1900-01-01';
SELECT * FROM master_dataset WHERE started_at > GETDATE();

### Handling Station ID Formats
Inconsistent `start_station_id` and `end_station_id` formats were identified and corrected.

In [ ]:
SELECT 
  CASE
    WHEN start_station_id LIKE '[A-Z]%' THEN 'Starts with Letter'
    WHEN start_station_id LIKE '[0-9]%' THEN 'Starts with Number'
    ELSE 'Other/Irregular'
  END AS FormatType,
  COUNT(*) AS Count
FROM master_dataset
GROUP BY 
  CASE
    WHEN start_station_id LIKE '[A-Z]%' THEN 'Starts with Letter'
    WHEN start_station_id LIKE '[0-9]%' THEN 'Starts with Number'
    ELSE 'Other/Irregular'
  END;

ALTER TABLE master_dataset 
ADD start_station_id_format_issue varchar (40);

UPDATE master_dataset
SET start_station_id_format_issue = 
  CASE
    WHEN start_station_id LIKE '[A-Z][A-Z][0-9]%' 
         AND start_station_id NOT LIKE '%[^A-Z0-9]%' 
         AND LEN(start_station_id) >= 8
      THEN 'OK'
    WHEN start_station_id LIKE '[0-9]%' THEN 'Number'
    ELSE 'Other'
  END;

ALTER TABLE master_dataset 
ADD end_station_id_format_issue varchar (40);

UPDATE master_dataset
SET end_station_id_format_issue = 
  CASE
    WHEN end_station_id LIKE '[A-Z][A-Z][0-9]%' 
         AND end_station_id NOT LIKE '%[^A-Z0-9]%' 
         AND LEN(end_station_id) >= 8
      THEN 'OK'
    WHEN end_station_id LIKE '[0-9]%' THEN 'Number'
    ELSE 'Other'
  END;

DELETE FROM master_dataset WHERE start_station_id_format_issue != 'OK' OR end_station_id_format_issue != 'OK';

ALTER TABLE master_dataset
DROP COLUMN start_station_id_format_issue, end_station_id_format_issue;

### Validating Coordinates
Coordinates with incorrect formats were checked and removed.

In [ ]:
SELECT *
FROM master_dataset
WHERE
    LEN(CAST(FLOOR(ABS(start_lat)) AS VARCHAR)) > 2
    OR LEN(CAST(FLOOR(ABS(end_lat)) AS VARCHAR)) > 2
    OR LEN(CAST(FLOOR(ABS(start_lng)) AS VARCHAR)) > 2
    OR LEN(CAST(FLOOR(ABS(end_lng)) AS VARCHAR)) > 2;

DELETE FROM master_dataset
WHERE
    FORMAT(start_lat, '0.0000000000000') LIKE '%0000000'
    OR FORMAT(end_lat,   '0.0000000000000') LIKE '%0000000'
    OR FORMAT(start_lng, '0.0000000000000') LIKE '%0000000'
    OR FORMAT(end_lng,   '0.0000000000000') LIKE '%0000000';

### Handling Typographical Errors
Columns were checked for typos and extra spaces, with trimming applied where necessary.

In [ ]:
SELECT DISTINCT member_casual, 
       LEN(member_casual) AS ActualLength,
       DATALENGTH(member_casual) AS ByteLength,
       '[' + member_casual + ']' AS ValuePreview
FROM master_dataset
ORDER BY member_casual;

UPDATE master_dataset
SET member_casual = LTRIM(RTRIM(member_casual));

UPDATE master_dataset
SET rideable_type = LTRIM(RTRIM(rideable_type));
GO
UPDATE Cyclistic_Case_Study.dbo.master_dataset
SET start_station_name = LTRIM(RTRIM(start_station_name));
GO
UPDATE Cyclistic_Case_Study.dbo.master_dataset
SET end_station_name = LTRIM(RTRIM(end_station_name));
GO
UPDATE Cyclistic_Case_Study.dbo.master_dataset
SET start_station_id = LTRIM(RTRIM(start_station_id));
GO
UPDATE Cyclistic_Case_Study.dbo.master_dataset
SET end_station_id = LTRIM(RTRIM(end_station_id));

### Correcting Station Name Typos
Specific typos in station names were corrected.

In [ ]:
SELECT DISTINCT start_station_name 
FROM Cyclistic_Case_Study.dbo.master_dataset 
ORDER BY start_station_name;
SELECT DISTINCT end_station_name 
FROM Cyclistic_Case_Study.dbo.master_dataset 
ORDER BY end_station_name;

UPDATE Cyclistic_Case_Study.dbo.master_dataset
SET end_station_name = 'Damen Ave & Thomas St' WHERE end_station_name='Damen Ave & Thomas St (Augusta Blvd)';
UPDATE Cyclistic_Case_Study.dbo.master_dataset
SET end_station_name = 'Damen Ave & Walnut St' WHERE end_station_name='Damen Ave & Walnut (Lake) St';

### Trip Duration Validation
Trips with invalid durations (zero, negative, or excessively long) were removed.

In [ ]:
SELECT *
FROM Cyclistic_Case_Study.dbo.master_dataset
WHERE DATEDIFF(MINUTE, started_at, ended_at) <= 0;

DELETE FROM Cyclistic_Case_Study.dbo.master_dataset
WHERE DATEDIFF(MINUTE, started_at, ended_at) <=0;

SELECT *
FROM Cyclistic_Case_Study.dbo.master_dataset
WHERE DATEDIFF(HOUR, started_at, ended_at) >= 24;

DELETE FROM Cyclistic_Case_Study.dbo.master_dataset
WHERE DATEDIFF(HOUR, started_at, ended_at) >= 24;

SELECT *
FROM Cyclistic_Case_Study.dbo.master_dataset
WHERE ended_at <= started_at;

### Removing Short Loop Trips
Trips where the start and end stations were the same with durations under 5 minutes were removed, as they likely represent test rides or errors.

In [ ]:
SELECT *
FROM Cyclistic_Case_Study.dbo.master_dataset
WHERE start_station_name = end_station_name
  AND DATEDIFF(MINUTE, started_at, ended_at) < 5
  AND start_station_name IS NOT NULL;

DELETE FROM Cyclistic_Case_Study.dbo.master_dataset
WHERE start_station_name = end_station_name
  AND DATEDIFF(MINUTE, started_at, ended_at) < 5;

## Summary of Data Cleaning Procedures
The following table summarizes the data cleaning steps performed on the Cyclistic dataset.

| Step | Description | Outcome |
|------|-------------|---------|
| Data Import | Imported 12 CSV files into separate SQL tables, consolidated into `master_dataset`. | 12 tables merged, `dataset_4` imported via `BULK INSERT` due to errors. |
| Remove Duplicates | Used CTE to delete duplicate `ride_id` entries. | Ensured unique `ride_id` values. |
| Set Primary Key | Set `ride_id` as primary key. | Enforced data integrity. |
| Add Indexes | Created indexes on `started_at`, `ended_at`, `start_station_id`, `end_station_id`, and `member_casual`. | Improved query performance. |
| Remove NULLs | Deleted rows with NULL values in critical columns. | Ensured complete data for analysis. |
| Validate Dates | Checked `started_at` and `ended_at` for anomalies (e.g., future dates, 1900-01-01). | No invalid dates found. |
| Fix Station IDs | Flagged and removed invalid `start_station_id` and `end_station_id` formats. | Ensured consistent station ID formats. |
| Validate Coordinates | Removed rows with invalid coordinates (e.g., excessive zeros). | Ensured valid geographic data. |
| Trim Spaces | Trimmed extra spaces from `member_casual`, `rideable_type`, `start_station_name`, `end_station_name`, `start_station_id`, `end_station_id`. | Standardized categorical data. |
| Fix Typos | Corrected station name typos (e.g., removed extra descriptors). | Improved station name consistency. |
| Validate Trip Duration | Removed trips with zero/negative durations (<3040 rows) and durations >24 hours (9 rows). | Eliminated invalid trips. |
| Remove Short Loops | Deleted trips with same start/end station and duration <5 minutes (>9000 rows). | Removed likely test rides or errors. |